In [ ]:
from scipy.io import wavfile
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset, label =  [], []
n = 1
l = 0
c = 0
for filename in os.listdir('/content/drive/MyDrive/namakav_waves'):
  audio = '/content/drive/MyDrive/namakav_waves/'+filename
  fs, data = wavfile.read(audio)
  b = 0
  duration = int(data.shape[0]/int(n*fs))
  for i in range (duration):
    left_CH = data[b:b+int(n*fs),0]
    right_CH = data[b:b+int(n*fs),1]
    L_CH = left_CH.reshape(left_CH.shape[0],1)
    dataset.append(L_CH)
    label.append(0)
    R_CH = right_CH.reshape(right_CH.shape[0],1)
    dataset.append(R_CH)
    label.append(1)
    b = b+int(n*fs)
  l = l + data.shape[0]/int(n*fs)
  c = c+1

print("number of wav files is : ",str(c))
print("sampling frequency of wav files is : ",str(fs)," hz")
print("average length of wav files is : ",str(l/c/60)," minutes")

In [ ]:
# Tensorflow / Keras
import tensorflow as tf # used to access argmax function
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout # for adding Concolutional and densely-connected NN layers.
from keras import layers

# Data manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
import sklearn # for model evaluation
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.preprocessing import OrdinalEncoder # for encoding labels

# Visualization
import cv2 # for ingesting images
import matplotlib
import matplotlib.pyplot as plt # for showing images

# Other utilities
import sys
import os

In [ ]:
# Convert audio data to numpy array
dataset = np.array(dataset, dtype="float")

# Show data shape
print("Shape of whole data: ", dataset.shape)

# Convert Labels list to numpy array
label=np.array(label)

# Encode labels
enc = OrdinalEncoder()
y=enc.fit_transform(label.reshape(-1,1))

In [ ]:

# ---- Create training and testing samples ---
train_ratio = 0.75
validation_ratio = 0.20
test_ratio = 0.05
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=1-train_ratio, random_state=0)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=0)
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

# Print shapes
# Note, model input must have a four-dimensional shape [samples, rows, columns, channels]
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)
print("Shape of X_val: ", X_val.shape)
print("Shape of y_val: ", y_val.shape)

In [ ]:
# first model without batch normalization
model_1 = Sequential()
model_1.add(Conv1D(8, 3, activation='relu', input_shape=(int(n*fs),1)))
model_1.add(Conv1D(16, 3, activation='relu'))
model_1.add(MaxPooling1D())


model_1.add(Conv1D(32, 3, activation='relu'))
model_1.add(Conv1D(64, 3, activation='relu'))
model_1.add(MaxPooling1D())

model_1.add(Conv1D(128, 3, activation='relu'))
model_1.add(Conv1D(64, 3, activation='relu'))
model_1.add(MaxPooling1D())

model_1.add(Conv1D(128, 3, activation='relu'))

model_1.add(Flatten())
model_1.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.0005)
model_1.compile(loss=tf.losses.BinaryCrossentropy(),optimizer=opt, metrics=['accuracy'])

model_1.summary()

hist_1 = model_1.fit(X_train, y_train, epochs=7, validation_data=(X_val, y_val))

In [ ]:
# sample model with more layers and without batch normalization
model_2 = Sequential()
model_2.add(Conv1D(8, 3, activation='relu', input_shape=(int(n*fs),1)))
model_2.add(Conv1D(16, 3, activation='relu'))
model_2.add(MaxPooling1D())


model_2.add(Conv1D(32, 3, activation='relu'))
model_2.add(Conv1D(64, 3, activation='relu'))
model_2.add(MaxPooling1D())

model_2.add(Conv1D(128, 3, activation='relu'))
model_2.add(Conv1D(64, 3, activation='relu'))
model_2.add(MaxPooling1D())

model_2.add(Conv1D(128, 3, activation='relu'))

model_2.add(Conv1D(128, 3, activation='relu'))
model_2.add(Conv1D(64, 3, activation='relu'))
model_2.add(MaxPooling1D())

model_2.add(Conv1D(128, 3, activation='relu'))


model_2.add(Flatten())
model_2.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.0005)
model_2.compile(loss=tf.losses.BinaryCrossentropy(),optimizer=opt, metrics=['accuracy'])

model_2.summary()

hist_2 = model_2.fit(X_train, y_train, epochs=7, validation_data=(X_val, y_val))

In [ ]:
model_3 = Sequential()
model_3.add(Conv1D(8, 3, activation='relu', input_shape=(int(n*fs),1)))

model_3.add(BatchNormalization())
model_3.add(Conv1D(16, 3, activation='relu'))
model_3.add(MaxPooling1D())


model_3.add(Conv1D(32, 3, activation='relu'))
model_3.add(Conv1D(64, 3, activation='relu'))
model_3.add(MaxPooling1D())


model_3.add(Conv1D(128, 3, activation='relu'))
model_3.add(Conv1D(64, 3, activation='relu'))
model_3.add(MaxPooling1D())


model_3.add(Conv1D(128, 3, activation='relu'))

model_3.add(Flatten())

model_3.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.0005)
model_3.compile(loss=tf.losses.BinaryCrossentropy(),optimizer=opt, metrics=['accuracy'])

model_3.summary()


hist_3 = model_3.fit(X_train, y_train, batch_size=32, epochs=7, validation_data=(X_val, y_val))

In [ ]:
plt.figure(figsize=(30,4))
plt.subplot(1,3,1)
plt.plot(hist_1.history['loss'], color='teal', label='loss')
plt.plot(hist_1.history['val_loss'], color='orange', label='val_loss')
plt.title('Loss model1', fontsize=20)
plt.legend(loc="upper left")

plt.subplot(1,3,2)
plt.plot(hist_2.history['loss'], color='teal', label='loss')
plt.plot(hist_2.history['val_loss'], color='orange', label='val_loss')
plt.title('Loss model2', fontsize=20)
plt.legend(loc="upper left")


plt.subplot(1,3,3)
plt.plot(hist_3.history['loss'], color='teal', label='loss')
plt.plot(hist_3.history['val_loss'], color='orange', label='val_loss')
plt.title('Loss model3', fontsize=20)
plt.legend(loc="upper left")

plt.show()

plt.figure(figsize=(30,4))
plt.subplot(1,3,1)
plt.plot(hist_1.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist_1.history['val_accuracy'], color='orange', label='val_accuracy')
plt.title('Accuracy model1', fontsize=20)
plt.legend(loc="upper left")

plt.subplot(1,3,2)
plt.plot(hist_2.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist_2.history['val_accuracy'], color='orange', label='val_accuracy')
plt.title('Accuracy model2', fontsize=20)
plt.legend(loc="upper left")

plt.subplot(1,3,3)
plt.plot(hist_3.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist_3.history['val_accuracy'], color='orange', label='val_accuracy')
plt.title('Accuracy model3', fontsize=20)
plt.legend(loc="upper left")


plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

yhat = model_1.predict(X_test)
pre.update_state(y_test, yhat)
re.update_state(y_test, yhat)
acc.update_state(y_test, yhat)

print("Test Part for model1")
print("Precision: ",pre.result().numpy(),"    ", "Recall: ", re.result().numpy(),"    ", "Accuracy: ",acc.result().numpy())


yhat = model_2.predict(X_test)
pre.update_state(y_test, yhat)
re.update_state(y_test, yhat)
acc.update_state(y_test, yhat)

print("Test Part for model2")
print("Precision: ",pre.result().numpy(),"    ", "Recall: ", re.result().numpy(),"    ", "Accuracy: ",acc.result().numpy())


yhat = model_3.predict(X_test)
pre.update_state(y_test, yhat)
re.update_state(y_test, yhat)
acc.update_state(y_test, yhat)


print("Test Part for model3")
print("Precision: ",pre.result().numpy(),"    ", "Recall: ", re.result().numpy(),"    ", "Accuracy: ",acc.result().numpy())

In [ ]:

def predictQualityofAudio(model,audio_dir,target):
    dataset = []
    fs, data = wavfile.read('/content/drive/MyDrive/'+audio_dir)
    n=1
    counter=0
    b = 0
    duration = int(data.shape[0]/int(n*fs))
    for i in range (duration):
      sample = data[b:b+int(n*fs)]
      sample = sample.reshape(sample.shape[0],1)
      dataset.append(sample)
      b = b+int(n*fs)
      counter+=1

    dataset = np.array(dataset, dtype="float")
    yhat = model.predict(dataset)
    if target==0:
      score = len(yhat[yhat<=0.5])
      print("The quality of predict is "+str(score)+" out of "+str(counter)+" for a valid sound "+audio_dir+".")
    elif target==1:
      score = len(yhat[yhat>=0.5])
      print("The quality of predict is "+str(score)+" out of "+str(counter)+ " for a damaged sound "+audio_dir+".")

print("model_1:")
predictQualityofAudio(model_1,"RMPABOL0.wav",0)
predictQualityofAudio(model_1,"RMPABOL1.wav",1)
predictQualityofAudio(model_1,"RMPBARZ0.wav",0)
predictQualityofAudio(model_1,"RMPBARZ1.wav",1)

print("model_2:")
predictQualityofAudio(model_2,"RMPABOL0.wav",0)
predictQualityofAudio(model_2,"RMPABOL1.wav",1)
predictQualityofAudio(model_2,"RMPBARZ0.wav",0)
predictQualityofAudio(model_2,"RMPBARZ1.wav",1)


print("model_3:")
predictQualityofAudio(model_3,"RMPBARZ0.wav",0)
predictQualityofAudio(model_3,"RMPBARZ1.wav",1)
predictQualityofAudio(model_3,"RMPABOL0.wav",0)
predictQualityofAudio(model_3,"RMPABOL1.wav",1)
